In [28]:
import os



In [29]:
%pwd

'd:\\AI\\projects\\DL_KIDNEY_DISEASE_CLASSIFICATION'

In [38]:
os.chdir(r"D:\AI\projects\DL_KIDNEY_DISEASE_CLASSIFICATION")

In [31]:
MLFLOW_TRACKING_URI=https://dagshub.com/shaikmaaheed/DL_KIDNEY_TUMOUR.mlflow \
MLFLOW_TRACKING_USERNAME=shaikmaaheed \
MLFLOW_TRACKING_PASSWORD= 3bb977c7bf158b13e478bdbd0b5a43cba6d8f636  \
python script.py

SyntaxError: invalid syntax (4244732624.py, line 1)

In [ ]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/shaikmaaheed/DL_KIDNEY_TUMOUR.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "shaikmaaheed"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "3bb977c7bf158b13e478bdbd0b5a43cba6d8f636"

In [32]:
import tensorflow as tf

In [39]:
from dataclasses import dataclass
from pathlib import Path


@dataclass()
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size:list
    params_batch_size: int

In [40]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)


        create_directories([self.config.artifact_root])

    def get_evaluation_config(self)->EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= "artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/shaikmaaheed/DL_KIDNEY_TUMOUR.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE

        )
        return eval_config

In [41]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [42]:
class Evaluation:
    def __init__(self, config:EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy":self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
   


    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")

            else:
                mlflow.keras.log_model(self.model, "model")





In [44]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    # logger.error(e)
    raise e

[2023-12-25 09:43:16,658 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2023-12-25 09:43:16,666 : INFO : common : yaml file: params.yaml loaded successfully]
[2023-12-25 09:43:16,671 : INFO : common : created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 16s 2s/step - loss: 4.3196 - accuracy: 0.5036


2023/12/25 09:43:35 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-12-25 09:43:37,394 : WARNING : save : Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\MAAHEED\AppData\Local\Temp\tmpevr1xf58\model\data\model\assets
[2023-12-25 09:43:38,064 : INFO : builder_impl : Assets written to: C:\Users\MAAHEED\AppData\Local\Temp\tmpevr1xf58\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2023/12/25 09:45:00 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
